In [2]:
import torch
import torch.nn as nn
import timm
import numpy as np
from PIL import Image
from torchvision import transforms

# Define your model class (same as training)
class EfficientNetForAnimalRecognition(nn.Module):
    def __init__(self, num_classes=1072):
        super().__init__()
        self.efficientnet = timm.create_model('efficientnet_b3', pretrained=False, num_classes=0)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(1536, num_classes)

    def forward(self, x):
        features = self.efficientnet.forward_features(x)
        features = self.global_pool(features)
        features = torch.flatten(features, 1)
        return self.classifier(features)

    def extract_features(self, x):
        features = self.efficientnet.forward_features(x)
        features = self.global_pool(features)
        return torch.flatten(features, 1)

# Image preprocessing (must match training)
def preprocess_image(img_path):
    transform = transforms.Compose([
        transforms.Resize((300, 300)),  # EfficientNet-B3 prefers 300x300
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                             std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(img_path).convert("RGB")
    return transform(image).unsqueeze(0)  # Add batch dimension

# Load model and weights
def load_model(path, device):
    model = EfficientNetForAnimalRecognition()
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    return model

# Extract normalized feature vector
def get_normalized_features(model, image_tensor, device):
    image_tensor = image_tensor.to(device)
    with torch.no_grad():
        features = model.extract_features(image_tensor)
        features = features / features.norm(dim=1, keepdim=True)  # Normalize to unit vector
    return features.cpu().numpy()




In [3]:
# Example usage
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = load_model("best_model.pth", device)
    
    image1 = preprocess_image("sample_img/2_a.jpg")
    image2 = preprocess_image("sample_img/2_b.jpg")

    feat1 = get_normalized_features(model, image1, device)
    feat2 = get_normalized_features(model, image2, device)

    from sklearn.metrics.pairwise import cosine_similarity
    similarity = cosine_similarity(feat1, feat2)

    print("Cosine Similarity:", similarity[0][0])

Cosine Similarity: 0.9254955
